[View in Colaboratory](https://colab.research.google.com/github/gmihaila/toy_models/blob/master/keras_generator.ipynb)

### Use data Generator in Keras

by GeorgeM.

In [2]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

--2018-05-14 16:52:30--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.007s  

2018-05-14 16:52:30 (3.12 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]



In [50]:
ls

datalab/  pima-indians-diabetes.data.csv


In [88]:
# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense
import time
import numpy as np

# fix random seed for reproducibility
np.random.seed(7)
# load pima indians dataset
dataset = np.loadtxt('pima-indians-diabetes.data.csv', delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
data_size = len(X)
print 'Data size: ', data_size

data = []
for x,y in zip(X,Y):
  data.append([x,y])
print 'Data created'

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


def gen(data, mini_batch):
    print('Generator initiated!\n')
    # read or split data
    data_size = len(data)
    x = []    # input batch examples
    y = []    # output batch examples
    
    for index, pair in enumerate(data):
      # append data        
      x.append(pair[0])
      y.append(pair[1])
      
      if (index+1)%mini_batch == 0:
#         print '\nUsed %s%% of data'%(int(float(index+1)/float(len(data)) * 100))
        x = np.array(x)
        y = np.array(y)
        # output data
        yield x, y
        x = []; y = []

Data size:  768
Data created


In [89]:
# tr_gen = gen(data, 1)
# s = time.time()
# model.fit_generator(generator=tr_gen, steps_per_epoch=768, epochs=1, max_queue_size=0, workers=0, verbose=2)
# e = time.time()
# print '\n\n Generator: ',(e-s)


# steps_per_epoch: batches of examples/examples form Generator to finish 1 epoch
# max_queue_size: Integer. Maximum size for the generator queue. Default 10.
# workers: Integer. Maximum number of processes to spin up when using process-based threading. Default to 1. 
         # 0, will execute the generator on the main thread.
# epochs: number of times model sees data



# Fit the model
s = time.time()
model.fit(X, Y, epochs=1, batch_size=1)
e = time.time()
print '\n\n Generator: ',(e-s)

# evaluate the model

scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

del model

Epoch 1/1
768/768 [==============================] - 3s 3ms/step - loss: 5.6245 - acc: 0.6510


 Generator:  2.95914387703
768/768 [==============================] - 1s 699us/step

acc: 65.10%
